In [1]:
import actonet
import biolqm

In [2]:
lqm = biolqm.load("http://ginsim.org/sites/default/files/Bladder_Model.zginml")
bn = biolqm.to_minibn(biolqm.booleanize(lqm)).simplify()
bn

Downloading http://ginsim.org/sites/default/files/Bladder_Model.zginml

AKT <- PI3K
ATM_b1 <- DNAdamage|(ATM_b1&ATM_b2)
ATM_b2 <- ATM_b1&DNAdamage&E2F1_b1
Apoptosis_b1 <- TP53|(Apoptosis_b1&Apoptosis_b2)|(E2F1_b1&E2F1_b2)
Apoptosis_b2 <- Apoptosis_b1&E2F1_b1&E2F1_b2
CDC25A <- (!CHEK1_2_b1&E2F1_b1&!RBL2)|(!CHEK1_2_b1&E2F3_b1&!RBL2)
CHEK1_2_b1 <- ATM_b1|(CHEK1_2_b1&CHEK1_2_b2)
CHEK1_2_b2 <- ATM_b1&CHEK1_2_b1&E2F1_b1
CyclinA <- (CDC25A&E2F1_b1&!RBL2&!p21CIP)|(CDC25A&E2F3_b1&!RBL2&!p21CIP)
CyclinD1 <- (AKT&!p16INK4a&!p21CIP)|(RAS&!p16INK4a&!p21CIP)
CyclinE1 <- (CDC25A&E2F1_b1&!RBL2&!p21CIP)|(CDC25A&E2F3_b1&!RBL2&!p21CIP)
DNAdamage <- DNAdamage
E2F1_b1 <- (E2F1_b1&E2F1_b2)|(E2F3_b1&!RB1&!RBL2)|(RAS&!RB1&!RBL2)
E2F1_b2 <- (ATM_b1&ATM_b2&CHEK1_2_b1&CHEK1_2_b2&E2F1_b1&E2F3_b1&E2F3_b2&!RB1&!RBL2)|(ATM_b1&ATM_b2&CHEK1_2_b1&CHEK1_2_b2&E2F1_b1&RAS&!RB1&!RBL2)
E2F3_b1 <- (E2F3_b1&E2F3_b2)|(RAS&!RB1)
E2F3_b2 <- CHEK1_2_b1&CHEK1_2_b2&E2F3_b1&RAS&!RB1
EGFR <- (EGFR_stimulus&!FGFR3&!GRB2)|(!FGFR3&!GRB2&SPRY)
EGFR_stimulus <- EGFR_stimulus
FGFR3 <- !EGFR&FGFR3_stimulus&!GRB

In [3]:
act = actonet.ActoNet(bn)

In [4]:
inputs = {"DNAdamage": 0, "FGFR3_stimulus": 1, "EGFR_stimulus": 0, "GrowthInhibitors": 1}
spec = {"Proliferation": 1, "Apoptosis_b1": 0, "Apoptosis_b2": 0, "Growth_Arrest": 0}

In [5]:
act.set_outputs(list(spec.keys()))
act.set_input_condition(inputs)
act.set_property(spec)
for sol in act.controls(maxcontrol=3):
    print(sol)

{'p16INK4a': 0, 'p21CIP': 0}
{'RBL2': 0, 'p21CIP': 0}
{'p16INK4a': 0, 'PI3K': 1}
{'p16INK4a': 0, 'AKT': 1}
{'p21CIP': 0, 'CyclinD1': 1}
{'RBL2': 0, 'AKT': 1}
{'RBL2': 0, 'EGFR': 1}
{'p16INK4a': 0, 'EGFR': 1}
{'CyclinD1': 1, 'EGFR': 1}
{'AKT': 1, 'CyclinD1': 1}
{'CyclinD1': 1, 'PI3K': 1}
{'RBL2': 0, 'PI3K': 1}
{'CyclinE1': 1}
{'RBL2': 0, 'GRB2': 1}
{'CyclinD1': 1, 'GRB2': 1}
{'p16INK4a': 0, 'GRB2': 1}


In [6]:
act.asp_fixpoint()

['vAKT :- holds(set(vAKT, 1)).',
 'vAKT :- not holds(set(vAKT, _)), not not vPI3K.',
 'vATM_b1 :- holds(set(vATM_b1, 1)).',
 'vATM_b1 :- not holds(set(vATM_b1, _)), holds(vDNAdamage).',
 'vATM_b1 :- not holds(set(vATM_b1, _)), not not vATM_b1, not not vATM_b2.',
 'vATM_b2 :- holds(set(vATM_b2, 1)).',
 'vATM_b2 :- not holds(set(vATM_b2, _)), not not vATM_b1, holds(vDNAdamage), not not vE2F1_b1.',
 'vApoptosis_b1 :- not not vTP53.',
 'vApoptosis_b1 :- not not vApoptosis_b1, not not vApoptosis_b2.',
 'vApoptosis_b1 :- not not vE2F1_b1, not not vE2F1_b2.',
 'vApoptosis_b2 :- not not vApoptosis_b1, not not vE2F1_b1, not not vE2F1_b2.',
 'vCDC25A :- holds(set(vCDC25A, 1)).',
 'vCDC25A :- not holds(set(vCDC25A, _)), not vCHEK1_2_b1, not not vE2F1_b1, not vRBL2.',
 'vCDC25A :- not holds(set(vCDC25A, _)), not vCHEK1_2_b1, not not vE2F3_b1, not vRBL2.',
 'vCHEK1_2_b1 :- holds(set(vCHEK1_2_b1, 1)).',
 'vCHEK1_2_b1 :- not holds(set(vCHEK1_2_b1, _)), not not vATM_b1.',
 'vCHEK1_2_b1 :- not holds(se

In [7]:
act.asp_guess()

['value(0;1).',
 'variable(vAKT; vATM_b1; vATM_b2; vApoptosis_b1; vApoptosis_b2; vCDC25A; vCHEK1_2_b1; vCHEK1_2_b2; vCyclinA; vCyclinD1; vCyclinE1; vDNAdamage; vE2F1_b1; vE2F1_b2; vE2F3_b1; vE2F3_b2; vEGFR; vEGFR_stimulus; vFGFR3; vFGFR3_stimulus; vGRB2; vGrowthInhibitors; vGrowth_Arrest; vMDM2; vPI3K; vPTEN; vProliferation; vRAS; vRB1; vRBL2; vSPRY; vTP53; vp14ARF; vp16INK4a; vp21CIP).',
 'output(vApoptosis_b1; vApoptosis_b2; vGrowth_Arrest; vProliferation).',
 'controlled(X) :- variable(X), not output(X).',
 'control(X,T) :- controlled(X), value(T).',
 '#const maxcontrol=2.',
 '{set(X,V) : control(X,V)} maxcontrol.',
 ':- set(X,0), set(X,1).',
 'holds(set(X,V)) :- set(X,V).',
 '#show holds/1.',
 '#show nholds/1.',
 '#project holds/1.',
 '{vFGFR3_stimulus; vDNAdamage; vGrowthInhibitors; vEGFR_stimulus}.',
 'holds(vFGFR3_stimulus) :- vFGFR3_stimulus.',
 'nholds(vFGFR3_stimulus) :- not holds(vFGFR3_stimulus).',
 'holds(vDNAdamage) :- vDNAdamage.',
 'nholds(vDNAdamage) :- not holds(vDNAd

In [8]:
act.asp_check()

[':- not vApoptosis_b1, not vApoptosis_b2, not vGrowth_Arrest, vProliferation.']